## Configurações de Banco de Dados

In [ ]:
MONGO_URI = "mongodb://localhost:27017/?replicaSet=rs0"
DB_NAME = "simlab_db"

## Iniciar um Novo Experimento

In [ ]:
from pymongo import MongoClient
import gridfs
from datetime import datetime
import os

client = MongoClient(MONGO_URI)
db = client[DB_NAME]

def insert_file(path: str, name: str):
    fs = gridfs.GridFS(db)
    # Armazenar um arquivo
    with open(path, "rb") as f:
        file_id = fs.put(f, filename=name, contentType="text/x-c")

    return file_id
    
def insert_experiment(
    experiment_data: dict,
    files: list
) -> str:
    """
    Insere um experimento no MongoDB seguindo o formato do modelo C#.
    
    Args:
        experiment_data: Dicionário com os dados do experimento no formato:
            {
                "Name": str,
                "Status": str (opcional, default="Waiting"),
                "StartTime": str (ISO format) ou datetime,
                "EndTime": str (ISO format) ou datetime ou None (opcional),
                "Parameters": List[Dict[str, str]] (ex: [{"Key": "param1", "Value": "value1"}]),
                "SimulationElements": Dict (conteúdo da simulação)
            }
    
    Returns:
        O _id do documento inserido
    """
    # Conecta ao MongoDB
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db["experiments"]
    
    # Prepara o documento com valores padrão
    document = {
        "Name": experiment_data.get("Name", ""),
        "Status": experiment_data.get("Status", "Waiting"),
        "StartTime": experiment_data["StartTime"],
        "Parameters": experiment_data.get("Parameters", []),
        "SimulationElements": experiment_data.get("SimulationElements", {})
    }
    
    paramenters = []
    for file in files:
        id = insert_file(file, os.path.basename(file))
        param = {
            "Name": file.get("Name",),
            "Type": "file",
            "Value": id
        }
        paramenters.append(param)
    
    # Adiciona EndTime se existir
    if "EndTime" in experiment_data and experiment_data["EndTime"] is not None:
        document["EndTime"] = experiment_data["EndTime"]
    
    # Converte strings para datetime se necessário
    if isinstance(document["StartTime"], str):
        document["StartTime"] = datetime.fromisoformat(document["StartTime"])
    
    if "EndTime" in document and isinstance(document["EndTime"], str):
        document["EndTime"] = datetime.fromisoformat(document["EndTime"])
    
    # Insere o documento no MongoDB
    result = collection.insert_one(document)
    
    # Fecha a conexão
    client.close()
    
    return str(result.inserted_id)

In [2]:
import os
from pymongo import MongoClient
import gridfs
from datetime import datetime
from typing import Optional

# Configurações do MongoDB (substitua conforme necessário)
MONGO_URI = "mongodb://localhost:27017"
DB_NAME = "experiments_db"

def insert_file(path: str, name: str) -> str:
    """
    Armazena um arquivo no GridFS e retorna o ID.
    
    Args:
        path: Caminho completo do arquivo no sistema de arquivos
        name: Nome do arquivo a ser armazenado
        
    Returns:
        ID do arquivo no GridFS
    """
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    fs = gridfs.GridFS(db)
    
    with open(path, "rb") as f:
        file_id = fs.put(f, filename=name)
    
    client.close()
    return str(file_id)

def insert_experiment(
    experiment_data: dict,
    file_parameters: Optional[list[dict]] = None
) -> str:
    """
    Insere um experimento no MongoDB seguindo o formato do modelo C# atualizado.
    
    Args:
        experiment_data: Dicionário com os dados do experimento no formato:
            {
                "Name": str,
                "Status": str (opcional, default="Waiting"),
                "StartTime": str (ISO format) ou datetime,
                "EndTime": str (ISO format) ou datetime ou None (opcional),
                "Parameters": List[Dict] (ex: [{"Name": "param1", "Type": "float", "Value": "1.0"}]),
                "SimulationElements": Dict (conteúdo da simulação)
            }
        file_parameters: Lista de dicionários com arquivos para upload via GridFS:
            [{
                "Name": "nome_do_parametro",
                "FilePath": "/caminho/para/arquivo.txt"
            }]
            
    Returns:
        O _id do documento inserido
    """
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db["experiments"]
    
    # Prepara o documento base
    document = {
        "Name": experiment_data.get("Name", ""),
        "Status": experiment_data.get("Status", "Waiting"),
        "StartTime": experiment_data["StartTime"],
        "Parameters": experiment_data.get("Parameters", []),
        "SimulationElements": experiment_data.get("SimulationElements", {})
    }
    
    # Processa arquivos via GridFS se fornecidos
    if file_parameters:
        for file_param in file_parameters:
            try:
                file_id = insert_file(file_param["FilePath"], file_param["Name"])
                param = {
                    "Name": file_param["Name"],
                    "Type": "file",
                    "Value": file_id  # Armazena o ID do GridFS
                }
                document["Parameters"].append(param)
            except Exception as e:
                print(f"Erro ao processar arquivo {file_param['FilePath']}: {str(e)}")
                continue
    
    # Adiciona EndTime se existir
    if "EndTime" in experiment_data and experiment_data["EndTime"] is not None:
        document["EndTime"] = experiment_data["EndTime"]
    
    # Converte strings para datetime se necessário
    if isinstance(document["StartTime"], str):
        document["StartTime"] = datetime.fromisoformat(document["StartTime"])
    
    if "EndTime" in document and isinstance(document["EndTime"], str):
        document["EndTime"] = datetime.fromisoformat(document["EndTime"])
    
    # Valida os parâmetros
    for param in document["Parameters"]:
        if not all(key in param for key in ["Name", "Type", "Value"]):
            raise ValueError("Cada parâmetro deve conter 'Name', 'Type' e 'Value'")
    
    # Insere o documento no MongoDB
    result = collection.insert_one(document)
    client.close()
    
    return str(result.inserted_id)

In [ ]:
import os
import json
from datetime import datetime
from typing import Dict, List, Optional

# Configurações
STATIC_DATA_DIR = "data"  # Diretório contendo os arquivos
SIM_ELEM_FILE = "simElem.json"   # Arquivo com os elementos de simulação
EXPERIMENT_NAME = "POC"

def load_simulation_elements(file_path: str) -> Dict:
    """Carrega os elementos de simulação de um arquivo JSON"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Arquivo {file_path} não encontrado. Usando valores padrão.")
        return {"Model": "Default Model", "Config": {}}
    except json.JSONDecodeError:
        print(f"Erro ao decodificar {file_path}. Verifique se é um JSON válido.")
        return {"Model": "Default Model", "Config": {}}

def get_files_from_directory(directory: str) -> List[Dict]:
    """Retorna lista de arquivos no diretório com seus caminhos completos"""
    file_params = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) and filename != SIM_ELEM_FILE:  # Ignora o arquivo de simulação
            # Remove a extensão do arquivo para usar como nome do parâmetro
            param_name = os.path.splitext(filename)[0]
            file_params.append({
                "Name": param_name,
                "FilePath": file_path
            })
    return file_params

# Carrega os elementos de simulação do arquivo JSON
simulation_elements = load_simulation_elements(SIM_ELEM_FILE)

# Obtém todos os arquivos do diretório static-data (exceto simElem.json)
file_parameters = get_files_from_directory(STATIC_DATA_DIR)

# Verifica se encontrou arquivos
if not file_parameters:
    print(f"Nenhum arquivo encontrado no diretório {STATIC_DATA_DIR}")
else:
    print(f"Encontrados {len(file_parameters)} arquivos para processar")

# Dados do experimento
experiment_data = {
    "Name": EXPERIMENT_NAME,
    "Status": "Enqueued",
    "StartTime": datetime.now().isoformat(),  # Usa o horário atual
    "Parameters": [],
    "SimulationElements": simulation_elements  # Usa os dados carregados do JSON
}

# Inserindo no MongoDB (se houver arquivos)
if file_parameters:
    inserted_id = insert_experiment(
        experiment_data=experiment_data,
        file_parameters=file_parameters
    )
    print(f"Documento inserido com ID: {inserted_id}")
    print(f"Total de arquivos processados: {len(file_parameters)}")
    print(f"Elementos de simulação carregados: {simulation_elements}")
else:
    print("Nenhum experimento foi inserido devido à falta de arquivos")